In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv('../data/2020_Notebook02_train_output.csv')
test = pd.read_csv('../data/2020_Notebook02_test_output.csv')

train

,latitud,longitud,Barrio,Ciudad,Area_total,Area_constr,MONEDA,Area_constr_cat,NroBanios,Dormitorios,...,Cocheras,areas_diff,areas_proporcion,geohash_grado5,geohash_grado6,geohash_grado7,geohash_grado8,geohash_grado9,diff_total_mean_district,Precio_m2_total
0,-12.119831,-77.016015,Miraflores,Lima,398.0,352.0,S/,11.0,4.0,4.0,...,2.0,46.0,1.130682,6mc5p,6mc5p3,6mc5p3w,6mc5p3ws,6mc5p3wsu,258.315217,6550.251256
1,-12.080734,-76.914384,La Molina,Lima,750.0,550.0,S/,17.0,4.0,4.0,...,4.0,200.0,1.363636,6mc73,6mc73b,6mc73bb,6mc73bb8,6mc73bb8s,205.376812,6030.000000
2,-12.089309,-76.935051,La Molina,Lima,1080.0,600.0,USD,18.0,4.0,4.0,...,4.0,480.0,1.800000,6mc71,6mc71r,6mc71r1,6mc71r1w,6mc71r1we,-27.605863,4212.962963
3,-12.106803,-77.051677,San Isidro,Lima,420.0,355.0,USD,12.0,3.0,4.0,...,3.0,65.0,1.183099,6mc5n,6mc5ns,6mc5nsh,6mc5nsh8,6mc5nsh8v,-98.233466,12250.000000
4,-12.093773,-77.067285,Magdalena,Lima,816.0,815.0,S/,19.0,5.0,7.0,...,3.0,1.0,1.001227,6mc5n,6mc5nq,6mc5nq3,6mc5nq3k,6mc5nq3kd,-398.451049,5147.058824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4702,-12.073589,-76.886133,La Molina,Lima,855.0,397.0,S/,13.0,4.0,4.0,...,3.0,458.0,2.153652,6mc76,6mc766,6mc766j,6mc766j4,6mc766j4p,-493.254902,2776.608187
4703,-12.354781,-76.809923,Punta Negra,Lima,437.0,340.0,S/,11.0,4.0,4.0,...,4.0,97.0,1.285294,6mc3s,6mc3sn,6mc3snu,6mc3snus,6mc3snusp,-303.451049,5242.105263
4704,-12.108089,-76.960098,Santiago de Surco,Lima,300.0,563.0,USD,18.0,4.0,4.0,...,4.0,-263.0,0.532860,6mc70,6mc70e,6mc70ez,6mc70ez3,6mc70ez37,691.766534,7583.333333
4705,-12.082020,-76.928234,La Molina,Lima,300.0,300.0,S/,9.0,4.0,4.0,...,2.0,0.0,1.000000,6mc73,6mc732,6mc732w,6mc732w9,6mc732w94,-258.233466,6640.000000


In [2]:
#Estas columnas se van a encodear con LabelEncoder.
#Para los Geohash utilizaremos otros metodo.

to_encoded_columns = train.select_dtypes(include='object').filter(regex='^(?!geohash)')

#Creamos los artifacts.
import pickle
from sklearn.preprocessing import LabelEncoder
label_encoder_dict = {}

for column in to_encoded_columns:
    label_encoder = LabelEncoder()
    label_encoder.fit(train[column])
    label_encoder_dict[column] = label_encoder    
    train[column] = label_encoder.transform(train[column])
    
    #Para encodear test también.
    test[column] = label_encoder.transform(test[column])
    
with open('../data/label_encoder_dict.pkl','wb') as handle:
    pickle.dump(label_encoder_dict, handle, protocol = pickle.HIGHEST_PROTOCOL)

#### Encodeo de Geohashes:

Para poder encodear los Geohashes tenemos que, como son variables de alta cardinalidad, tenemos que buscar un método para poder asignar geohashes cercanos a coordenadas nuevas que aparezcan cuando se haga la inferencia. 

In [5]:
train.drop('MONEDA',axis=1,inplace=True)
test.drop('MONEDA',axis=1,inplace=True)

In [6]:
#Por ahora no usaré geohashes. Veremos primero cómo va el performance con un modelo simple.
train.to_csv('../data/2020_Notebook03_train_output.csv',index=False)
test.to_csv('../data/2020_Notebook03_test_output.csv', index=False)

tree = BallTree(test_points, metric = 'haversine')

results = tree.query_radius(test_points, r=10, return_distance=True)


#From stackoverflow
from sklearn.neighbors import BallTree

ratio_tierra = 6371000
ratio_test = 1300000

test_points = [[-12.011108,-77.053447],[-12.003141, -77.06475]]
test_points_rad = np.array([[np.radians(x[0]), np.radians(x[1])] for x in test_points])

tree = BallTree(test_points_rad, metric='haversine')

ind, results = tree.query_radius(test_points_rad, r = ratio_test/ratio_tierra, return_distance=True)

print(ind)
print(results * ratio_tierra/1000)


def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Para calcular la distancia entre dos ubicaciones en la tierra.
    En kilometros.
    """
    r = 6371
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2
    res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
    return np.round(res, 4)

haversine_distance(-12.011108,-77.053447,-12.003141, -77.06475)


from sklearn.neighbors import BallTree
import numpy as np

rng = np.random.RandomState(1)
X = rng.random_sample((10,2))

tree = BallTree(X, leaf_size=1, metric='haversine')
dist, ind = tree.query(X[:1], k=2)
print(ind)
print(dist)